In [641]:
#TAREA 3 (INTEGRACIÓN) - MÉTODOS COMPUTACIONALES (JUAN PABLO MARTÍNEZ Y SANTIAGO RODRÍGUEZ)

In [642]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import integrate

In [643]:
#Ejercicio 3.
a = 0
b = 1
funcion = lambda x: np.e**(-x**2)
#f"(x) alcanza un valor absoluto maximo de 2 para el intervalo [0,1] en ξ=0
#Por lo tanto, [(1-0)^3 / 12n^2]*2 = 0.005
#Resolviendo para n, n=5.77. Osea tomamos n=6
n = 6
x = np.linspace(a,b,n+1)
class Integrator:
    def __init__(self,x,y,n):
        self.x=x
        self.y=y
        self.h=(x[-1]-x[0])/n
        self.integral = 0.
class TrapezoidMethod(Integrator):
    def __init__(self,x,y,n):
        Integrator.__init__(self,x,y,n)
    def GetIntegral(self):
        self.integral = 0.
        self.integral += 0.5*( self.y[0]+self.y[-1] )
        self.integral += np.sum( self.y[1:-1] )
        return self.integral*self.h

integrador = TrapezoidMethod(x,funcion(x),n)
print(f"La integral definida por método del trapecio con el error máximo dado es igual a {integrador.GetIntegral()}")





La integral definida por método del trapecio con el error máximo dado es igual a 0.7451194124361793


In [644]:
#Ejercicio 6.
#Definimos la función
R = 0.5
a = 0.01
inductancia = lambda x: np.sqrt(a**2-x**2)/(x+R)
#Primero, método del trapecio
n=36
x=np.linspace(-a,a,n+1)
integrador6 = TrapezoidMethod(x, inductancia(x), n)
inductancia_trapecio = integrador6.GetIntegral()
inductancia_real = np.pi*(R-np.sqrt(R**2-a**2))
error_trapecio = np.abs((inductancia_trapecio-inductancia_real)/inductancia_real)*100
print(f"El resultado de la estimación con método del trapecio fue {inductancia_trapecio} y el valor real es {inductancia_real}")
print(f"Es decir,el error fue de {round(error_trapecio,2)}%")

El resultado de la estimación con método del trapecio fue 0.0003126527987679914 y el valor real es 0.0003141906875702543
Es decir,el error fue de 0.49%


In [645]:
#Ahora, usamos Simpson 1/3
n_simpson = 2 #Subintervalo par y en 1/3 debe ser 2
x_simpson=np.linspace(-a,a,n_simpson+1)
class SimpsonSimple(Integrator):
    def __init__(self,x,y,n):
        Integrator.__init__(self,x,y,n)
    def GetIntegral(self):
        self.integral = 0.
        self.integral += self.y[0] + 4*self.y[1:-1][0] + self.y[-1] #Se hace el segundo slicing porque solo son 3 valores
        #Entonces [1:-1] da el valor entre el primero y el ultimo, ya que solo hay uno en la mitad.
        return self.integral*(self.h/3)
inductancia_simpson = SimpsonSimple(x_simpson, inductancia(x_simpson),n_simpson).GetIntegral()
error_simpson = np.abs((inductancia_simpson-inductancia_real)/inductancia_real)*100
inductancia_simpson
print(f"El resultado de la estimación con método de 1/3 simpson fue {inductancia_simpson} y el valor real es {inductancia_real}")
print(f"Es decir,el error fue de {round(error_simpson,2)}%")


El resultado de la estimación con método de 1/3 simpson fue 0.0002666666666666667 y el valor real es 0.0003141906875702543
Es decir,el error fue de 15.13%
